# InceptionV3 + LSTM video classifier

## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparing the dataset

### Dividing into training, test and validation

In [ ]:
# pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd '/content/drive/MyDrive/fight-detection'

/content/drive/MyDrive/fight-detection


In [ ]:
# import splitfolders 
# splitfolders.ratio('data/data_og', output="data", seed=2023, ratio=(.7, 0.2,0.1))

Copying files: 300 files [00:09, 31.67 files/s]


## Cloning the LSTM-video-classification repository

In [ ]:
%cd data

/content/drive/MyDrive/fight-detection/data


In [ ]:
!python extract_files.py mp4

Streaming output truncated to the last 5000 lines.
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100 / 55.  5.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'train/fight/fi012.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp41isom
    creation_time   : 2019-02-21T12:17:02.000000Z
  Duration: 00:00:03.25, start: 0.000000, bitrate: 699 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 640x360 [SAR 1:1 DAR 16:9], 526 kb/s, 20 fps, 20 tbr, 20k tbn, 40 tbc (default)
    Metadata:
      creation_time   : 2019-02-21T12:17:02.000000Z
      handler_name    : VideoHandler
      encoder         : AVC Coding
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 166 kb/s (default)
 

The file data/data_file.csv contains the duration (number of frames) of the videos. To classify all of them choose for the training a sequence lenght of the minimum duration

## Training

In [ ]:
import pandas as pd
df = pd.read_csv('data_file.csv')

In [ ]:
df = df.rename(columns={'63': 'seq_len'})
df.head()

,train,noFight,nofi039,seq_len
0,train,noFight,nofi023,63
1,train,noFight,nofi086,36
2,train,noFight,nofi029,48
3,train,noFight,nofi056,50
4,train,noFight,nofi070,42


In [ ]:
df.seq_len.describe()

count    299.000000
mean      57.046823
std       15.955827
min       20.000000
25%       50.000000
50%       60.000000
75%       60.000000
max      142.000000
Name: seq_len, dtype: float64

In [ ]:
for i in range(video.shape[0]):
      frame = cv2.resize(video[i], (128,128), interpolation=cv2.INTER_CUBIC)
      L.append(frame)

In [ ]:
import glob
import os
import cv2

folders = ['train', 'test', 'val']
extenssion = 'jpg'

for folder in folders:
    class_folders = glob.glob(os.path.join(folder, '*'))
    print(class_folders)

    for vid_class in class_folders:
        class_files = glob.glob(os.path.join(vid_class, '*.' + extenssion))
        print(class_files)

        for video_path in class_files:
          img = cv2.imread(video_path, cv2.IMREAD_UNCHANGED)
          resized = cv2.resize(img, (128,128), interpolation=cv2.INTER_CUBIC)
          cv2.imwrite(video_path, resized)



```
! python train.py 43 2 480 640
```



In [ ]:
%cd ..

/content/drive/MyDrive/fight-detection


In [ ]:
!python train.py 20 2 128 128

Streaming output truncated to the last 5000 lines.
data type: features
sample: ['train', 'fight', 'fi097', '60']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi038', '60']
Get the sequence from disk
data type: features
sample: ['train', 'fight', 'fi122', '52']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi011', '59']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi133', '60']
Get the sequence from disk
data type: features
sample: ['train', 'fight', 'fi124', '60']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi073', '42']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi106', '60']
Get the sequence from disk
data type: features
sample: ['train', 'fight', 'fi145', '60']
Get the sequence from disk
data type: features
sample: ['train', 'noFight', 'nofi089', '36']
Get the sequence from disk
data type: features
sample: ['train

## Evaluating

```
! python predict.py 43 2 lstm_features.hdf5 train
```



In [ ]:
!python predict.py 20 2  'data/checkpoints/lstm-features.001-0.858.hdf5'  train 480 640
 

2023-03-22 14:18:13.265544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 14:18:14.375442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-03-22 14:18:14.375561: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/u

## Clasifying

```
! python clasify.py 43 2 lstm-features.hdf5 forehand.avi
```



In [ ]:
!python /content/drive/MyDrive/cnn/Fork/LSTM-video-classification/clasify.py 43 2  /content/data/checkpoints/lstm-features.040-0.101.hdf5 /content/drive/MyDrive/cnn/Fork/LSTM-video-classification/videos/reves-4.avi